<a href="https://colab.research.google.com/github/aj1365/ResUNetFormer/blob/main/ResU_NetFormer_Het.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 # For CV operations
from PIL import Image  #To create and store images
import numpy as np

#To binarize the input
import h5py
import os
from patchify import patchify

In [ ]:
##### Creating input & mask arrays


images = []
originalImages = os.listdir("E:/MRD/tiff/train/")

for index,image in enumerate(originalImages):
    print("Image number : " +str(index) )
    img = Image.open("E:/MRD/tiff/train/" + str(image))
    img = img.resize((384, 384))
    arr = np.array(img)
    #arr = np.expand_dims(arr, -1)
    images.append(arr)

TrainX=images
TrainX = np.array(TrainX)



images = []
originalImages = os.listdir("E:/MRD/tiff/train_labels/")

for index,image in enumerate(originalImages):
    print("Image number : " +str(index) )
    img = Image.open("E:/MRD/tiff/train_labels/" + str(image))
    img = img.resize((384, 384))
    arr = np.array(img)
    #arr = np.expand_dims(arr, -1)
    images.append(arr)

TrainY=images
TrainY = np.array(TrainY)


In [ ]:
images = []
originalImages = os.listdir("E:/MRD/tiff/test/")

for index,image in enumerate(originalImages):
    print("Image number : " +str(index) )
    img = Image.open("E:/MRD/tiff/test/" + str(image))
    img = img.resize((384, 384))
    arr = np.array(img)
    #arr = np.expand_dims(arr, -1)
    images.append(arr)

TestX=images
TestX = np.array(TestX)
TestX.shape

images = []
originalImages = os.listdir("E:/MRD/tiff/test_labels/")

for index,image in enumerate(originalImages):
    print("Image number : " +str(index) )
    img = Image.open("E:/MRD/tiff/test_labels/" + str(image))
    img = img.resize((384, 384))
    arr = np.array(img)
    #arr = np.expand_dims(arr, -1)
    images.append(arr)

TestY=images
TestY = np.array(TestY)


In [ ]:
TrainX=TrainX[0:800,:,:,:]
TrainY=TrainY[0:800,:,:]

In [ ]:
TrainY=TrainY.reshape(TrainY.shape[0],TrainY.shape[1],TrainY.shape[1],1)
TestY=TestY.reshape(TestY.shape[0],TestY.shape[1],TestY.shape[1],1)

TrainY.shape, TestY.shape

In [ ]:
##############Finalizing Dataset for Training#######

with h5py.File("E:/Dataset_train.h5", 'w') as hdf:
    hdf.create_dataset('images', data=TrainX, compression='gzip', compression_opts=9)
    hdf.create_dataset('masks', data=TrainY, compression='gzip', compression_opts=9)

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras import backend as keras
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow.keras.backend as K
from typing import Callable
from keras_cv_attention_models.attention_layers import (
    activation_by_name,
    ChannelAffine,
    conv2d_no_bias,
    depthwise_conv2d_no_bias,
    drop_block,
    #MixupToken,
    mlp_block,
    output_block,
    add_pre_post_process,
)
from keras_cv_attention_models.download_and_load import reload_model_weights
from keras_cv_attention_models.attention_layers import (
    ChannelAffine,
    CompatibleExtractPatches,
    conv2d_no_bias,
    drop_block,
    layer_norm,
    mlp_block,
    output_block,
    add_pre_post_process,
)
from keras_cv_attention_models.download_and_load import reload_model_weights


In [ ]:
# Metrics to be used when evaluating the network
from tensorflow_addons.metrics import F1Score

precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
f1 = F1Score(num_classes=1, name='f1', average='micro', threshold=0.4)
sgd_optimizer = Adam()

In [ ]:
import math
import tensorflow_addons as tfa

tfk = tf.keras
tfkl = tfk.layers
tfm = tf.math
L2_WEIGHT_DECAY = 1e-4

In [ ]:
class MultiHeadRelativePositionalKernelBias(tf.keras.layers.Layer):
    def __init__(self, input_height=-1, is_heads_first=False, **kwargs):
        super().__init__(**kwargs)
        self.input_height, self.is_heads_first = input_height, is_heads_first

    def build(self, input_shape):
        # input (is_heads_first=False): `[batch, height * width, num_heads, ..., size * size]`
        # input (is_heads_first=True): `[batch, num_heads, height * width, ..., size * size]`
        blocks, num_heads = (input_shape[2], input_shape[1]) if self.is_heads_first else (input_shape[1], input_shape[2])
        size = int(tf.math.sqrt(float(input_shape[-1])))
        height = self.input_height if self.input_height > 0 else int(tf.math.sqrt(float(blocks)))
        width = blocks // height
        pos_size = 2 * size - 1
        initializer = tf.initializers.truncated_normal(stddev=0.02)
        self.pos_bias = self.add_weight(name="positional_embedding", shape=(num_heads, pos_size * pos_size), initializer=initializer, trainable=True)

        idx_hh, idx_ww = tf.range(0, size), tf.range(0, size)
        coords = tf.reshape(tf.expand_dims(idx_hh, -1) * pos_size + idx_ww, [-1])
        bias_hh = tf.concat([idx_hh[: size // 2], tf.repeat(idx_hh[size // 2], height - size + 1), idx_hh[size // 2 + 1 :]], axis=-1)
        bias_ww = tf.concat([idx_ww[: size // 2], tf.repeat(idx_ww[size // 2], width - size + 1), idx_ww[size // 2 + 1 :]], axis=-1)
        bias_hw = tf.expand_dims(bias_hh, -1) * pos_size + bias_ww
        bias_coords = tf.expand_dims(bias_hw, -1) + coords
        bias_coords = tf.reshape(bias_coords, [-1, size**2])[::-1]  # torch.flip(bias_coords, [0])

        bias_coords_shape = [bias_coords.shape[0]] + [1] * (len(input_shape) - 4) + [bias_coords.shape[1]]
        self.bias_coords = tf.reshape(bias_coords, bias_coords_shape)  # [height * width, 1 * n, size * size]
        if not self.is_heads_first:
            self.transpose_perm = [1, 0] + list(range(2, len(input_shape) - 1))  # transpose [num_heads, height * width] -> [height * width, num_heads]

    def call(self, inputs):
        if self.is_heads_first:
            return inputs + tf.gather(self.pos_bias, self.bias_coords, axis=-1)
        else:
            return inputs + tf.transpose(tf.gather(self.pos_bias, self.bias_coords, axis=-1), self.transpose_perm)

    def get_config(self):
        base_config = super().get_config()
        base_config.update({"input_height": self.input_height, "is_heads_first": self.is_heads_first})
        return base_config


def LWA(
    inputs, kernel_size=7, num_heads=4, key_dim=0, out_weight=True, qkv_bias=True, out_bias=True, attn_dropout=0, output_dropout=0, name=None
):
    _, hh, ww, cc = inputs.shape
    key_dim = key_dim if key_dim > 0 else cc // num_heads
    qk_scale = 1.0 / (float(key_dim) ** 0.5)
    out_shape = cc
    qkv_out = num_heads * key_dim

    should_pad_hh, should_pad_ww = max(0, kernel_size - hh), max(0, kernel_size - ww)
    if should_pad_hh or should_pad_ww:
        inputs = tf.pad(inputs, [[0, 0], [0, should_pad_hh], [0, should_pad_ww], [0, 0]])
        _, hh, ww, cc = inputs.shape

    qkv = keras.layers.Dense(qkv_out * 3, use_bias=qkv_bias, name=name and name + "qkv")(inputs)
    query, key_value = tf.split(qkv, [qkv_out, qkv_out * 2], axis=-1)  # Matching weights from PyTorch
    query = tf.expand_dims(tf.reshape(query, [-1, hh * ww, num_heads, key_dim]), -2)  # [batch, hh * ww, num_heads, 1, key_dim]

    # key_value: [batch, height // kernel_size, width // kernel_size, kernel_size, kernel_size, key + value]
    key_value = CompatibleExtractPatches(sizes=kernel_size, strides=1, padding="VALID", compressed=False)(key_value)
    padded = (kernel_size - 1) // 2
    # torch.pad 'replicate'
    key_value = tf.concat([tf.repeat(key_value[:, :1], padded, axis=1), key_value, tf.repeat(key_value[:, -1:], padded, axis=1)], axis=1)
    key_value = tf.concat([tf.repeat(key_value[:, :, :1], padded, axis=2), key_value, tf.repeat(key_value[:, :, -1:], padded, axis=2)], axis=2)

    key_value = tf.reshape(key_value, [-1, kernel_size * kernel_size, key_value.shape[-1]])
    key, value = tf.split(key_value, 2, axis=-1)  # [batch * block_height * block_width, kernel_size * kernel_size, key_dim]
    key = tf.transpose(tf.reshape(key, [-1, key.shape[1], num_heads, key_dim]), [0, 2, 3, 1])  # [batch * hh*ww, num_heads, key_dim, kernel_size * kernel_size]
    key = tf.reshape(key, [-1, hh * ww, num_heads, key_dim, kernel_size * kernel_size])  # [batch, hh*ww, num_heads, key_dim, kernel_size * kernel_size]
    value = tf.transpose(tf.reshape(value, [-1, value.shape[1], num_heads, key_dim]), [0, 2, 1, 3])
    value = tf.reshape(value, [-1, hh * ww, num_heads, kernel_size * kernel_size, key_dim])  # [batch, hh*ww, num_heads, kernel_size * kernel_size, key_dim]
    # print(f">>>> {query.shape = }, {key.shape = }, {value.shape = }")

    # [batch, hh * ww, num_heads, 1, kernel_size * kernel_size]
    attention_scores = keras.layers.Lambda(lambda xx: tf.matmul(xx[0], xx[1]))([query, key]) * qk_scale
    attention_scores = MultiHeadRelativePositionalKernelBias(input_height=hh, name=name and name + "pos")(attention_scores)
    attention_scores = keras.layers.Softmax(axis=-1, name=name and name + "attention_scores")(attention_scores)
    attention_scores = keras.layers.Dropout(attn_dropout, name=name and name + "attn_drop")(attention_scores) if attn_dropout > 0 else attention_scores

    # attention_output = [batch, block_height * block_width, num_heads, 1, key_dim]
    attention_output = keras.layers.Lambda(lambda xx: tf.matmul(xx[0], xx[1]))([attention_scores, value])
    attention_output = tf.reshape(attention_output, [-1, hh, ww, num_heads * key_dim])
    # print(f">>>> {attention_output.shape = }, {attention_scores.shape = }")

    if should_pad_hh or should_pad_ww:
        attention_output = attention_output[:, : hh - should_pad_hh, : ww - should_pad_ww, :]

    if out_weight:
        # [batch, hh, ww, num_heads * key_dim] * [num_heads * key_dim, out] --> [batch, hh, ww, out]
        attention_output = keras.layers.Dense(out_shape, use_bias=out_bias, name=name and name + "output")(attention_output)
    attention_output = keras.layers.Dropout(output_dropout, name=name and name + "out_drop")(attention_output) if output_dropout > 0 else attention_output
    return attention_output

In [ ]:

################################## LIBRARIES ##################################

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Conv2DTranspose, concatenate, Lambda, UpSampling2D
from tensorflow.keras import Model, Input
from contextlib import redirect_stdout


############################# CONVOLUTIONAL BLOCK #############################

def  HetConv(feature_map, conv_filter, kernel_size , strides):

    # Groupwise Convolution
    x1=Conv2D(filters=conv_filter, kernel_size=(3,3), groups=3, strides=strides, padding='same')(feature_map)

    # Pointwise Convolution
    x2= Conv2D(filters=conv_filter, kernel_size=(1,1), strides=strides, padding='same')(feature_map)


    addition = Add()([x1, x2])

    return addition


def conv_block(feature_map):

    # Main Path
    conv_1 = HetConv(feature_map, conv_filter=66, kernel_size=(3,3), strides=(1,1))
    bn = BatchNormalization()(conv_1)
    relu = Activation(activation='relu')(bn)
    conv_2 = HetConv(relu,conv_filter=66, kernel_size=(3,3), strides=(1,1))

    res_conn = HetConv(feature_map,conv_filter=66, kernel_size=(1,1), strides=(1,1))
    res_conn = BatchNormalization()(res_conn)
    addition = Add()([res_conn, conv_2])

    return addition


############################### RESIDUAL BLOCK ################################

def res_block(feature_map, conv_filter, stride):

    bn_1 = BatchNormalization()(feature_map)
    relu_1 = Activation(activation='relu')(bn_1)
    conv_1 = HetConv(relu_1, conv_filter, kernel_size=(3,3), strides=stride[0])



    bn_2 = BatchNormalization()(conv_1)
    relu_2 = Activation(activation='relu')(bn_2)
    conv_2 = HetConv(relu_2, conv_filter, kernel_size=(3,3), strides=stride[1])


    res_conn = HetConv(feature_map, conv_filter, kernel_size=(1,1), strides=stride[0])
    res_conn = BatchNormalization()(res_conn)
    addition = Add()([res_conn, conv_2])

    return addition

################################### ENCODER ###################################

def encoder(feature_map):

    # Initialize the to_decoder connection
    to_decoder = []

    # Block 1 - Convolution Block
    path = conv_block(feature_map)
    to_decoder.append(path)

    # Block 2 - Residual Block 1
    path = res_block(path, 126, [(2, 2), (1, 1)])
    to_decoder.append(path)

    # Block 3 - Residual Block 2
    path = res_block(path, 252, [(2, 2), (1, 1)])
    to_decoder.append(path)

    return to_decoder

################################### DECODER ###################################

def decoder(feature_map, from_encoder):

    # Block 1: Up-sample, Concatenation + Residual Block 1
    main_path = UpSampling2D(size=(2,2), interpolation='bilinear')(feature_map)
    # main_path = Conv2DTranspose(filters=256, kernel_size=(2,2), strides=(2,2), padding='same')(feature_map)
    main_path = concatenate([main_path, from_encoder[2]], axis=3)
    main_path = res_block(main_path, 252, [(1, 1), (1, 1)])

    # Block 2: Up-sample, Concatenation + Residual Block 2
    main_path = UpSampling2D(size=(2,2), interpolation='bilinear')(main_path)
    # main_path = Conv2DTranspose(filters=128, kernel_size=(2,2), strides=(2,2), padding='same')(main_path)
    main_path = concatenate([main_path, from_encoder[1]], axis=3)
    main_path = res_block(main_path, 126, [(1, 1), (1, 1)])

    # Block 3: Up-sample, Concatenation + Residual Block 3
    main_path = UpSampling2D(size=(2,2), interpolation='bilinear')(main_path)
    # main_path = Conv2DTranspose(filters=64, kernel_size=(2,2), strides=(2,2), padding='same')(main_path)
    main_path = concatenate([main_path, from_encoder[0]], axis=3)
    main_path = res_block(main_path, 66, [(1, 1), (1, 1)])

    return main_path

################################ RESIDUAL UNET ################################
n=1
attn_kernel_size=3
num_heads=4
attn_drop_rate=0.1
hidden_size=256

def ResLWAUNet():

    # Input
    x = Input(shape=(384, 384, 3))
   # model_input_float = Lambda(lambda x: x / 255)(model_input)


    # Encoder Path
    model_encoder = encoder(x)
    model_bottleneck = res_block(model_encoder[2], 510, [(2, 2), (1, 1)])
    # Transformer/Encoder

    y= LWA(model_bottleneck,
                attn_kernel_size,
                num_heads,
                attn_dropout=attn_drop_rate,
                name=f"Transformer/encoderblock_{n}")


    # Bottleneck

    # Decoder Path
    model_decoder = decoder(y, model_encoder)

    # Output

    output_layer = Conv2D(filters=1, kernel_size=(1, 1), strides=(1, 1), activation='sigmoid', padding='same')(model_decoder)


    model=Model(inputs=x, outputs=output_layer)
    model.compile(optimizer=sgd_optimizer, loss='binary_crossentropy', metrics=['accuracy', precision, recall, f1])


    return model

In [ ]:
model=ResLWAUNet()
model.summary()

In [ ]:
print('*'*30)
print('Loading and preprocessing train data...')
print('*'*30)
file = h5py.File('E:/Dataset_train.h5', 'r')
imgs_train = file.get('images')
imgs_mask_train = file.get('masks')
imgs_train = np.array(imgs_train)
imgs_mask_train = np.array(imgs_mask_train)

print(imgs_train.shape)
print(imgs_mask_train.shape)


imgs_train = imgs_train.astype('float32')

mean = np.mean(imgs_train)  # mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std

imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255  # scale masks to [0, 1]

print('*'*30)
print('Creating and compiling model...')
print('*'*30)
model = ResLWAUNet()

In [ ]:
weight_decay = 0.0001
learning_rate=1e-4

optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )


checkpoint_filepath = "E:/MRD100/ResUNetFormer.h5"



#with tf.device('/CPU:0'):
history = model.fit(
        x=imgs_train,
        y=imgs_mask_train,
        batch_size=1,
        epochs=20,
        validation_split=0.1
    )


In [ ]:
model.save('E:/MRD100/ResUNetFormer.h5')

In [ ]:
###### Creating Test Dataset
testImages=TestX

testImages.shape

with h5py.File("E:/Dataset_test.h5", 'w') as hdf:
    hdf.create_dataset('images', data=testImages, compression='gzip', compression_opts=9)



file = h5py.File('E:/Dataset_test.h5', 'r')
imgs_test = file.get('images')
#imgs_mask_test = file.get('masks')
imgs_test = np.array(imgs_test)
#imgs_mask_test = np.array(imgs_mask_test)
imgs_test = imgs_test.astype('float32')
imgs_test -= mean
imgs_test /= std

print('*'*30)
print('Loading saved weights...')
print('*'*30)
model.load_weights('E:/MRD100/ResUNetFormer.h5')

print('*'*30)
print('Predicting masks on test data...')
print('*'*30)
imgs_mask_test = model.predict(imgs_test, verbose=1,batch_size=1)
imgs_mask_test=(imgs_mask_test - np.min(imgs_mask_test))/(np.max(imgs_mask_test) - np.min(imgs_mask_test))
imgs_mask_test = (imgs_mask_test * 255).astype(np.uint8)

#imgs_mask_test = (imgs_mask_test * 255).astype(np.uint8)

In [ ]:
acc = model.evaluate(imgs_test, TestY, batch_size=1)

In [ ]:
######################### Write the predicted images

print('*' * 30)
print('Saving predicted masks to files...')
print('*' * 30)
pred_dir = 'E:/PredictionsResUNetFormer'

if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
for i, image in enumerate(imgs_mask_test):
    #image = (image * 255).astype(np.uint8)

    image=(image - np.min(image))/(np.max(image) - np.min(image))
    image = (image * 255).astype(np.uint8)

    cv2.imwrite(os.path.join(pred_dir, str(i + 1) + '_pred.png'), image)